# Loading acquisition parameters and setting up an acquisition scheme

## Setting up a Mipy acquisition scheme by loading acquisition parameters

The first thing to do is to load the parameters of some acquisition scheme. Typically, the b-values and gradient directions, along with the used pulse duration time $\delta$ and pulse separation time $\Delta$ are known. The Microstruktur toolbox uses SI units, so be careful, as bvalues are typically saved in $s/mm^2$, but we need them in $s/m^2$.

As an example we load the acquisition parameters of the WU-MINN Human Connectome Project:

In [1]:
# load the necessary modules
from mipy.core import modeling_framework
from mipy.acquisition_scheme.acquisition_scheme import acquisition_scheme_from_bvalues
from os.path import join
import numpy as np

# the HCP acquisition parameters are saved in the following toolbox path:
acquisition_path = modeling_framework.GRADIENT_TABLES_PATH

# we can then load the parameters themselves and convert them to SI units:
bvalues = np.loadtxt(join(acquisition_path, 'bvals_hcp_wu_minn.txt'))  # given in s/mm^2
bvalues_SI = bvalues * 1e6  # now given in SI units as s/m^2
gradient_directions = np.loadtxt(join(acquisition_path, 'bvecs_hcp_wu_minn.txt'))  # on the unit sphere

# The delta and Delta times we know from the HCP documentation in seconds
delta = 0.0106  
Delta = 0.0431 

# The acquisition scheme we use in the toolbox is then simply created as follows:
acq_scheme = acquisition_scheme_from_bvalues(bvalues_SI, gradient_directions, delta, Delta)

From the acquisition scheme, some basic information can be called using the following command:

In [2]:
acq_scheme.print_acquisition_info

Acquisition scheme summary

total number of measurements: 288
number of b0 measurements: 18
number of DWI shells: 3

shell_index |# of DWIs |bvalue [s/mm^2] |gradient strength [mT/m] |delta [ms] |Delta[ms]
0           |18        |0               |0                        |10.6       |43.1 
1           |90        |1000            |56                       |10.6       |43.1 
2           |90        |2000            |79                       |10.6       |43.1 
3           |90        |3000            |97                       |10.6       |43.1 


As the summary shows, the function automatically separates the different acquisition shells and detects b0 measurements.  
Other metrics such as qvalues, gradient strengths, diffusion times and the shell_indices can also be explicitly called as follows:

In [3]:
acq_scheme.bvalues;  # bvalues in s/m^2
acq_scheme.gradient_directions;  # gradient directions on the unit sphere
acq_scheme.gradient_strengths;  # the gradient strength in T/m
acq_scheme.qvalues;  # describes the diffusion sensitization in 1/m
acq_scheme.tau;  # diffusion time as Delta - delta / 3. in seconds
acq_scheme.shell_indices;  # index assigned to each shell. 0 is assigned to b0 measurements

## Setting up a Mipy acquisition scheme by converting a Dipy GradientTable

Dipy's GradientTable and Mipy's AcquisitionScheme both store and verify a dataset's acquisition parameters to later be used in fitting some model. For convenience, we provide the possibility to convert a dipy table to a mipy scheme using a simple conversion function.

In [4]:
from dipy.core.gradients import gradient_table
from mipy.acquisition_scheme.acquisition_scheme import gtab_dipy2mipy
gtab_dipy = gradient_table(bvalues, gradient_directions, big_delta=Delta, small_delta=delta)
acq_scheme_mipy = gtab_dipy2mipy(gtab_dipy)
acq_scheme_mipy.print_acquisition_info

Acquisition scheme summary

total number of measurements: 288
number of b0 measurements: 18
number of DWI shells: 3

shell_index |# of DWIs |bvalue [s/mm^2] |gradient strength [mT/m] |delta [ms] |Delta[ms]
0           |18        |0               |0                        |10.6       |43.1 
1           |90        |1000            |56                       |10.6       |43.1 
2           |90        |2000            |79                       |10.6       |43.1 
3           |90        |3000            |97                       |10.6       |43.1 


As you can see, the result is exactly the same. Note that the conversion requires that big_delta and small_delta are both set in the gradient_table!